In [ ]:
from utils import CNC
from types import SimpleNamespace

In [ ]:
args = SimpleNamespace(
    file_path = "./data/data.csv",
    model = "NB",
    over_sampling = True,
    future_steps = 1,
    window_size = 1,
    test_size = 200,
    threshold = 0.5,
    seed = 0,
)
cnc = CNC(args)

In [ ]:
cnc.pre_process()

In [ ]:
cnc.train()

In [ ]:
cnc.evaluate()

In [ ]:
cnc.visualize()

In [ ]:
import pandas as pd

raw = pd.read_csv("./data/data.csv")
new = pd.read_csv("./data/data_改過.csv")
raw['RPM'] = new['RPM']
raw.to_csv("./data/data.csv", index=False)